In [ ]:
'''At the beginning of this assignment, I had no experience with APIs, Flask, JS, Databases or creating a program outside of Python. 
This is what I was able to accomplish in five days of self-study and two false-starts. I understandthat this is not an acceptable version 
of the app that was requested, nor am I satisfied with my own final product. That said,I felt that it was better
to send what I had completely in the time given, as a closer on this assignment as well as a marker of my current abilities.

I just began learning how to code in August. Next semester, I will be taking classes in database management, back-end web architecture,
and advanced data structures. Which to say, I will be better. Now that I have encountered these concepts, I intend to continue improving 
my ability to organize them elegantly and creatively to design more complex systems.

Thank you for providing me with my first opportunity to explore these concepts. I would please ask that you not judge this too harshly
against my future potential. Should an internship position still be available during the Spring, and you feel that I am worth taking another
shot on, then I look forward to showing you what I will have learned by then.
-Max
'''


'''***PROGRAM START***'''


import requests
import random
import mysql.connector

'''
API Call: Used to pull in our API values and then add them to a mySQL database

Notes: This was my first time ever working with APIs. I considered making each ingredient an object,
but decided it would be better to store it as a tuple because then I could use the executemany function of mySQL
when populating my table. This later led to problems with my Random Taco function, so it might be better to classify
them once they are first entered.
'''
#used to iterate through each ingredient and build a list
api_call = [
    '/shells/',
    '/baseLayers/',
    '/mixins/',
    '/condiments/',
    '/seasonings/',   
]

ingredient_list = [] #list to hold our ingredients and categories
for i in range(len(api_call)):
    response = requests.get("https://ct-tacoapi.azurewebsites.net" + api_call[i]) #iterate through each endpoint
    #print(response.status())
    ingred = response.json() #converts api into string
    for j in ingred:        
        ingredient_list.append((j['name'],api_call[i].replace('/', ''))) #creates tuple for each ingredient with name and type for executemany

#print(ingredient_list) #used for testing
    
'''
DATABASE Connections

Notes: This was also my first time working with databases, though I was familiar with SQL queries from my workplace.
While it would be unlikely that I would add the code to create the table in the actual code, this served as a good primer
for understanding how to begin making changes. Next semester I will be taking a database management class, so I expect to be 
much more familiar with it.
'''

#again, would almost definitely not put in the hardcoded values into my final version on Flask for security concerns,
#but this made it easier to establish a connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="ChristmasTime",
    database="History_Schema"
)

mycursor = mydb.cursor()


'''code to create a table for the list of Tacos created by the user. Each row would hold a description (generated by the 
Taco object(see below)), as well as any ingredients). '''

#mycursor.execute("CREATE TABLE Tacos (Description VARCHAR(255), Shell VARCHAR(255), Base VARCHAR(255), Mixins VARCHAR(255), Condiments VARCHAR(255), Seasonings VARCHAR(255))")


'''code to create a table for the ingredients pulled in by the API. Each row would hold the name of the ingredient, as well
 as the type'''

#mycursor.execute("CREATE TABLE Ingredients (Name VARCHAR (255), Type VARCHAR(255))")

ingredientInsert = "INSERT INTO Ingredients (Name, Type) VALUES (%s, %s)"
mycursor.executemany(ingredientInsert, ingredient_list) #this uses our API list from before to populate our ingredients
#table with everything we've pulled. Now, we have a large table we can pull from based on the Type column to display
#multiple tables from the same dataset


'''SQL querys to pull ingredient tables'''
#shell table
#mycursor.execute("SELECT * FROM Ingredients WHERE type = 'shells'")


#base layers table
#mycursor.execute("SELECT * FROM Ingredients WHERE type = 'baseLayers'")


#mixins table
#mycursor.execute("SELECT * FROM Ingredients WHERE type = 'mixins'")


#condiments table
#mycursor.execute("SELECT * FROM Ingredients WHERE type = 'condiments'")


#seasonings table
#mycursor.execute("SELECT * FROM Ingredients WHERE type = 'seasonings'")


#tacos table
#mycursor.execute("SELECT * FROM Tacos")

'''
Python Functions & Classes

Notes:
'''

class Taco:
    def __init__ (self, shell, base, mixins, condiments, seasonings):
        self.shell = str(shell)
        self.base = str(base)
        self.mixins = str(mixins)
        self.condiments = str(condiments)
        self.seasonings = str(seasonings)

    
    def __str__(self): #in this case I decided to make the string function of the class served as the auto-generated description of the taco
        adjective_list = ['zesty', 'juicy', 'spicy', 'flavorful', 'scrumptous', 'bitchin'] #used for taco adjectives
        print("A " + random.choice(adjective_list) + " " + self.shell + " taco with " + self.base + ", " + self.mixins + ", " + self.condiments + ", " + self.seasonings)

        
'''
Functions:

Notes: I intended to use these in a helper file that I could call from my Flask code.
'''        
        
def create_taco(ingredient_list):
    
    #ideally this would be a form in Flask that would allow users to select ingredients from each table. In the interim,
    #I decided I would make it a basic text menu          
    print('Welcome! What kind of taco would you like?')    
    choice1 = ingred_choice('shells', ingredient_list) #calls ingred function below
    print(choice1) #testing
    choice2 = ingred_choice('baseLayers', ingredient_list)
    print(choice2) #testing
    choice3 = ingred_choice('mixins', ingredient_list)
    print(choice3) #testing
    choice4 = ingred_choice('condiments', ingredient_list)
    print(choice4) #testing
    choice5 = ingred_choice('seasonings', ingredient_list)
    print(choice5) #testing
    newtaco = Taco(choice1, choice2, choice3, choice4, choice5) #creates taco objectusing choices
    
    
    #calls database to insert row into table (I would have added an id in the final database so that I could call it when removing
    #a taco in the future, but something for next time)
    TacoInsert = "INSERT INTO Tacos (Description, Shell, Base, Mixins, Condiments, Seasonings) VALUES (%s, %s, %s, %s, %s, %s)"
    taco = (newtaco.__str__(), newtaco.shell, newtaco.base, newtaco.mixins, newtaco.condiments, newtaco.seasonings)
    mycursor.execute(TacoInsert,taco)

def ingred_choice(itype, ingredient_list): #used for iterating through each ingredient and building a list to select from
    choice_list = []
    print('What kind of ' + str(itype) + ' do you want?')
    for i in ingredient_list: 
        if i[1] == itype: #checks if its the type we need
            choice_list.append(i)
            print(str(len(choice_list)) + '. ' + str(i[0]))
    while True:
        try:
            choice = int(input('> ')) - 1 #gives us the index position of the list we built
            return choice_list[choice][0] #returns the name of the ingredient
        except:
            print('please input the number next to the ' + str(itype) + ' you want')    


'''Stretch Goal 1: Remove Taco
The best way to do this is to make sure the mySQL table has a column for id in it. That way when we pass in our newly created
taco object in the create_taco function, we can pass in the object id to the table. For removing it, we can then pull the id
of the taco we want to remove and pass that in to the SQL Query. At the moment, my table does not have a column for id,
so the next best thing is going to be to remove the last taco in the sql table
'''    
def remove_last_taco():
    #not working because we don't have an id we can query to be removed 
    
#     taco_nuke = input('please enter the id of the Taco you would like eliminated')
#     delete_taco = "DELETE FROM Tacos WHERE id = %s"
#     mycursor.execute(delete_taco, (taco_nuke,)) 

    #second best
        
    delete_last = "DELETE FROM Tacos ORDER BY datetime DESC LIMIT 1"
    mycursor.execute(delete_last) 

def menu(ingredient_list):
    print('what would you like to do?')
    print('a. Make a taco')
    print('b. Remove most recent taco')
    print('q. Quit')
    while True:
        try:
            choice = str(input('> '))
            if choice == 'a':
                create_taco(ingredient_list)
            elif choice =='b':
                remove_last_taco(ingredient_list)
            elif choice =='q':
                break
            else:
                print("I'm sorry, I didn't understand that, please input something else")
        except:
            print("I'm sorry, I didn't understand that, please input something else")
    
    
    
create_taco(ingredient_list)


mydb.commit()


In [ ]:
'''FLASK CODE
Notes: This was also the first time I was able to work with this. I ended up having the least amount of time to spend on
this, but I will be taking back-end web architecture in the spring semester and expect my 